<a href="https://colab.research.google.com/github/vajihe-ameri/predict-software-bugs-in-java-classes/blob/main/tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install requirements

In [1]:
!pip install sklearn pandas
!pip install prettytable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=a77bb40833bf96b8f5ee0654037fb10d9b14583f65f6b70c0feb5b8c7aeba9be
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Read dataset

In [2]:
import pandas as pd
from google.colab import drive 
drive = drive.mount('/content/drive') 

Mounted at /content/drive


Call algorithms from sklearntrain_featurestrain_fea

In [3]:
from prettytable import PrettyTable
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.tree import export_graphviz
from IPython import display
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [5]:
test_features = pd.read_csv("/content/drive/MyDrive/new_df/test_set.csv");
test_features 

,CC,CCL,CCO,CI,CLC,CLLC,LDC,LLDC,LCOM5,NL,...,Java Logging Rules,JavaBean Rules,Migration Rules,Naming Rules,Optimization Rules,Security Code Guideline Rules,Strict Exception Rules,String and StringBuffer Rules,Type Resolution Rules,Unnecessary and Unused Code Rules
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010309,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0
1,0.102759,0.012821,0.001800,0.010638,0.105769,0.123596,0.018676,0.019281,0.103093,0.012195,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.048077,0.014493,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030928,0.012195,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.014493,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010309,0.012195,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.014493,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010309,0.012195,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041237,0.012195,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0
4995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010309,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0
4996,0.294554,0.128205,0.011251,0.132979,0.189135,0.243455,0.079796,0.081507,0.010309,0.012195,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0
4997,0.074334,0.012821,0.000900,0.005319,0.050505,0.062112,0.008489,0.008764,0.010309,0.085366,...,0.0,0.0,0.0,0.006494,0.0,0.25,0.000000,0.000000,0.000000,0.0


In [6]:
train_features = pd.read_csv("/content/drive/MyDrive/new_df/train_set.csv");
train_features

,CC,CCL,CCO,CI,CLC,CLLC,LDC,LLDC,LCOM5,NL,...,Java Logging Rules,JavaBean Rules,Migration Rules,Naming Rules,Optimization Rules,Security Code Guideline Rules,Strict Exception Rules,String and StringBuffer Rules,Type Resolution Rules,Unnecessary and Unused Code Rules
0,0.131297,0.038462,0.008101,0.026596,0.120253,0.133838,0.048387,0.046450,0.061856,0.146341,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.228571,0.019231,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020619,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.009615,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020619,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.014493,0.000000
3,0.046410,0.012821,0.000900,0.005319,0.029499,0.039062,0.008489,0.008764,0.041237,0.060976,...,0.0,0.0,0.0,0.006494,0.02439,0.000000,0.000000,0.000000,0.000000,0.034483
4,0.234491,0.076923,0.005851,0.058511,0.188139,0.203046,0.078098,0.070114,0.020619,0.024390,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.028571,0.153846,0.159420,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19990,0.337975,0.064103,0.006301,0.026596,0.149837,0.221675,0.039049,0.039439,0.010309,0.024390,...,0.0,0.0,0.0,0.084416,0.02439,0.166667,0.000000,0.000000,0.000000,0.000000
19991,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020619,0.024390,...,0.0,0.0,0.0,0.032468,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
19992,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010309,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
19993,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103093,0.060976,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
test_target = pd.read_csv("/content/drive/MyDrive/new_df/test_set_NB.csv");
test_target

,Number of Bugs
0,0
1,0
2,0
3,0
4,2
...,...
4994,1
4995,0
4996,2
4997,0


In [8]:
train_target = pd.read_csv("/content/drive/MyDrive/new_df/train_set_NB.csv");
train_target 

,Number of Bugs
0,1
1,1
2,0
3,1
4,1
...,...
19990,2
19991,1
19992,0
19993,2


random forest

In [9]:
print('Random forest')

from sklearn.ensemble import RandomForestClassifier
'''Create random forest classifier'''
model1 = RandomForestClassifier()
model2 = RandomForestClassifier(n_estimators=100, criterion = 'gini', max_depth = 1, random_state = 80)
model3 = RandomForestClassifier(n_estimators=90, criterion = 'entropy' , max_depth = 10, random_state = 50)
model4 = RandomForestClassifier(n_estimators=50, criterion = 'entropy', max_depth = 20, random_state = 60)
model5 = RandomForestClassifier(n_estimators=1, criterion = 'gini', max_depth = 30, random_state = 40)

'''Train and test'''
model1.fit(train_features, train_target)
model2.fit(train_features, train_target)
model3.fit(train_features, train_target)
model4.fit(train_features, train_target)
model5.fit(train_features, train_target)

preds1 = model1.predict(test_features)
preds2 = model2.predict(test_features)
preds3 = model3.predict(test_features)
preds4 = model4.predict(test_features)
preds5 = model5.predict(test_features)

cr1 = confusion_matrix(test_target, preds1)
cr2 = confusion_matrix(test_target, preds2)
cr3 = confusion_matrix(test_target, preds3)
cr4 = confusion_matrix(test_target, preds4)
cr5 = confusion_matrix(test_target, preds5)

from sklearn.metrics import f1_score

f1_weighted1 = f1_score(test_target, preds1, average = 'weighted')
f1_weighted2 = f1_score(test_target, preds2, average = 'weighted')
f1_weighted3 = f1_score(test_target, preds3, average = 'weighted')
f1_weighted4 = f1_score(test_target, preds4, average = 'weighted')
f1_weighted5 = f1_score(test_target, preds5, average = 'weighted')

f1_macro1 = f1_score(test_target, preds1, average = 'macro')
f1_macro2 = f1_score(test_target, preds2, average = 'macro')
f1_macro3 = f1_score(test_target, preds3, average = 'macro')
f1_macro4 = f1_score(test_target, preds4, average = 'macro')
f1_macro5 = f1_score(test_target, preds5, average = 'macro')

f1_micro1 = f1_score(test_target, preds1, average = 'micro')
f1_micro2 = f1_score(test_target, preds2, average = 'micro')
f1_micro3 = f1_score(test_target, preds3, average = 'micro')
f1_micro4 = f1_score(test_target, preds4, average = 'micro')
f1_micro5 = f1_score(test_target, preds5, average = 'micro')

f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)
f1_score_weighted5 = "%.2f"%round(f1_weighted5, 2)

f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
f1_score_macro4 = "%.2f"%round(f1_macro4, 2)
f1_score_macro5 = "%.2f"%round(f1_macro5, 2)

f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
f1_score_micro4 = "%.2f"%round(f1_micro4, 2)
f1_score_micro5 = "%.2f"%round(f1_micro5, 2)

head = PrettyTable(['random_state', 'n_estimatorsint', 'criterion', 'max_depth', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro']) 
head.add_row(['', '', '', '', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1]) 
head.add_row(['80', '10', 'gini', '1', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2]) 
head.add_row(['50', '100', 'entropy', '10', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3]) 
head.add_row(['60', '50', 'entropy', '20', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4]) 
head.add_row(['40', '1', 'gini', '30', cr5, f1_score_weighted5, f1_score_macro5, f1_score_micro5]) 
print(head)

Random forest


ValueError: ignored

logistic regression

In [ ]:
print('Logistic regression')
from sklearn.linear_model import LogisticRegression

'''Create logistic regression classifier'''  
model1 = LogisticRegression()
model2 = LogisticRegression(solver='sag', penalty='l2',n_jobs=-1, random_state=10)
model3 = LogisticRegression(solver='saga', penalty='l2',n_jobs=-1, random_state=20)
model4 = LogisticRegression(solver='liblinear', penalty='l1',n_jobs=-1, random_state=0)
model5 = LogisticRegression(solver='liblinear', penalty='l2',n_jobs=-1, random_state=90)
model6 = LogisticRegression(solver='lbfgs', penalty='l2',n_jobs=-1, random_state=50)
model7 = LogisticRegression(solver='newton-cg', penalty='l2',n_jobs=-1, random_state=90)

'''Train and test'''
model1.fit(train_features, train_target)
model2.fit(train_features, train_target)
model3.fit(train_features, train_target)
model4.fit(train_features, train_target)
model5.fit(train_features, train_target)
model6.fit(train_features, train_target)
model7.fit(train_features, train_target)

preds1 = model1.predict(test_features)
preds2 = model2.predict(test_features)
preds3 = model3.predict(test_features)
preds4 = model4.predict(test_features)
preds5 = model5.predict(test_features)
preds6 = model5.predict(test_features)
preds7 = model5.predict(test_features)

cr1 = confusion_matrix(test_target, preds1)
cr2 = confusion_matrix(test_target, preds2)
cr3 = confusion_matrix(test_target, preds3)
cr4 = confusion_matrix(test_target, preds4)
cr5 = confusion_matrix(test_target, preds5)
cr6 = confusion_matrix(test_target, preds5)
cr7 = confusion_matrix(test_target, preds5)

from sklearn.metrics import f1_score

f1_weighted1 = f1_score(test_target, preds1, average = 'weighted')
f1_weighted2 = f1_score(test_target, preds2, average = 'weighted')
f1_weighted3 = f1_score(test_target, preds3, average = 'weighted')
f1_weighted4 = f1_score(test_target, preds4, average = 'weighted')
f1_weighted5 = f1_score(test_target, preds5, average = 'weighted')
f1_weighted6 = f1_score(test_target, preds5, average = 'weighted')
f1_weighted7 = f1_score(test_target, preds5, average = 'weighted')

f1_macro1 = f1_score(test_target, preds1, average = 'macro')
f1_macro2 = f1_score(test_target, preds2, average = 'macro')
f1_macro3 = f1_score(test_target, preds3, average = 'macro')
f1_macro4 = f1_score(test_target, preds4, average = 'macro')
f1_macro5 = f1_score(test_target, preds5, average = 'macro')
f1_macro6 = f1_score(test_target, preds5, average = 'macro')
f1_macro7 = f1_score(test_target, preds5, average = 'macro')

f1_micro1 = f1_score(test_target, preds1, average = 'micro')
f1_micro2 = f1_score(test_target, preds2, average = 'micro')
f1_micro3 = f1_score(test_target, preds3, average = 'micro')
f1_micro4 = f1_score(test_target, preds4, average = 'micro')
f1_micro5 = f1_score(test_target, preds5, average = 'micro')
f1_micro6 = f1_score(test_target, preds5, average = 'micro')
f1_micro7 = f1_score(test_target, preds5, average = 'micro')
 
f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)
f1_score_weighted5 = "%.2f"%round(f1_weighted5, 2)
f1_score_weighted6 = "%.2f"%round(f1_weighted6, 2)
f1_score_weighted7 = "%.2f"%round(f1_weighted7, 2)

f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
f1_score_macro4 = "%.2f"%round(f1_macro4, 2)
f1_score_macro5 = "%.2f"%round(f1_macro5, 2)
f1_score_macro6 = "%.2f"%round(f1_macro6, 2)
f1_score_macro7 = "%.2f"%round(f1_macro7, 2)

f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
f1_score_micro4 = "%.2f"%round(f1_micro4, 2)
f1_score_micro5 = "%.2f"%round(f1_micro5, 2)
f1_score_micro6 = "%.2f"%round(f1_micro6, 2)
f1_score_micro7 = "%.2f"%round(f1_micro7, 2)

head = PrettyTable(['random_state', 'solver', 'penalty', 'n_jobs', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro']) 
head.add_row(['', '', '', '', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1]) 
head.add_row(['10', 'sag', 'l2', '-1', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2]) 
head.add_row(['20', 'saga', 'l2', '-1', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3]) 
head.add_row(['0', 'liblinear', 'l1', '-1', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4]) 
head.add_row(['90', 'liblinear', 'l2', '-1', cr5, f1_score_weighted5, f1_score_macro5, f1_score_micro5]) 
head.add_row(['50', 'lbfgs', 'l2', '-1', cr6, f1_score_weighted6, f1_score_macro6, f1_score_micro6]) 
head.add_row(['90', 'newton-cg', 'l2', '-1', cr7, f1_score_weighted7, f1_score_macro7, f1_score_micro7]) 
print(head)

Logistic regression


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver

+--------------+-----------+---------+--------+------------------+-------------------+----------------+----------------+
| random_state |   solver  | penalty | n_jobs | confusion_matrix | f1_score_weighted | f1_score_macro | f1_score_micro |
+--------------+-----------+---------+--------+------------------+-------------------+----------------+----------------+
|              |           |         |        |    [[123   6]    |        0.80       |      0.52      |      0.83      |
|              |           |         |        |    [ 19   2]]    |                   |                |                |
|      10      |    sag    |    l2   |   -1   |    [[127   2]    |        0.79       |      0.46      |      0.85      |
|              |           |         |        |    [ 21   0]]    |                   |                |                |
|      20      |    saga   |    l2   |   -1   |    [[129   0]    |        0.80       |      0.46      |      0.86      |
|              |           |    

naive bayes

In [ ]:
print('Naive bayes')
from sklearn.naive_bayes import GaussianNB

'''Create Naive bayes classifier'''
model1 = GaussianNB()
model2 = GaussianNB(var_smoothing = 1e-11)
model3 = GaussianNB(var_smoothing = 1e-09)
model4 = GaussianNB(var_smoothing = 1e-10)

'''Train and test'''
model1.fit(train_features, train_target)
model2.fit(train_features, train_target)
model3.fit(train_features, train_target)
model4.fit(train_features, train_target)

preds1 = model1.predict(test_features)
preds2 = model2.predict(test_features)
preds3 = model3.predict(test_features)
preds4 = model4.predict(test_features)

cr1 = confusion_matrix(test_target, preds1)
cr2 = confusion_matrix(test_target, preds2)
cr3 = confusion_matrix(test_target, preds3)
cr4 = confusion_matrix(test_target, preds4)

from sklearn.metrics import f1_score

f1_weighted1 = f1_score(test_target, preds1, average = 'weighted')
f1_weighted2 = f1_score(test_target, preds2, average = 'weighted')
f1_weighted3 = f1_score(test_target, preds3, average = 'weighted')
f1_weighted4 = f1_score(test_target, preds4, average = 'weighted')

f1_macro1 = f1_score(test_target, preds1, average = 'macro')
f1_macro2 = f1_score(test_target, preds2, average = 'macro')
f1_macro3 = f1_score(test_target, preds3, average = 'macro')
f1_macro4 = f1_score(test_target, preds4, average = 'macro')

f1_micro1 = f1_score(test_target, preds1, average = 'micro')
f1_micro2 = f1_score(test_target, preds2, average = 'micro')
f1_micro3 = f1_score(test_target, preds3, average = 'micro')
f1_micro4 = f1_score(test_target, preds4, average = 'micro')

f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)

f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
f1_score_macro4 = "%.2f"%round(f1_macro4, 2)

f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
f1_score_micro4 = "%.2f"%round(f1_micro4, 2)

head = PrettyTable(['var_smoothing', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro']) 
head.add_row(['', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1]) 
head.add_row(['1e-11', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2]) 
head.add_row(['1e-9', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3]) 
head.add_row(['1e-10', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4]) 
print(head)

Naive bayes
+---------------+------------------+-------------------+----------------+----------------+
| var_smoothing | confusion_matrix | f1_score_weighted | f1_score_macro | f1_score_micro |
+---------------+------------------+-------------------+----------------+----------------+
|               |     [[89 53]     |        0.74       |      0.50      |      0.65      |
|               |     [ 0  8]]     |                   |                |                |
|     1e-11     |     [[58 84]     |        0.56       |      0.37      |      0.44      |
|               |     [ 0  8]]     |                   |                |                |
|      1e-9     |     [[89 53]     |        0.74       |      0.50      |      0.65      |
|               |     [ 0  8]]     |                   |                |                |
|     1e-10     |     [[69 73]     |        0.63       |      0.42      |      0.51      |
|               |     [ 0  8]]     |                   |                |     

Multi-Layer Perceptron Neural Network

In [ ]:
print('Multi layer perceptron neural network')
from sklearn.neural_network import MLPClassifier

'''Create Artificial Neural Network classifier'''
model1 = MLPClassifier()
model2 = MLPClassifier(solver='lbfgs', alpha=0.01, hidden_layer_sizes=(5, 2), activation = 'tanh', random_state=10)
model3 = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(150, 100, 50), activation = 'logistic',random_state=1)
model4 = MLPClassifier(solver='adam', alpha=0.05, hidden_layer_sizes=(100, 50, 30), activation = 'relu', random_state=90)
model5 = MLPClassifier(solver='lbfgs', alpha=0.0001, hidden_layer_sizes=(120, 80, 40), activation = 'identity', random_state=40)

'''Train and test'''
model1.fit(train_features, train_target)
model2.fit(train_features, train_target)
model3.fit(train_features, train_target)
model4.fit(train_features, train_target)
model5.fit(train_features, train_target)

preds1 = model1.predict(test_features)
preds2 = model2.predict(test_features)
preds3 = model3.predict(test_features)
preds4 = model4.predict(test_features)
preds5 = model5.predict(test_features)

cr1 = confusion_matrix(test_target, preds1)
cr2 = confusion_matrix(test_target, preds2)
cr3 = confusion_matrix(test_target, preds3)
cr4 = confusion_matrix(test_target, preds4)
cr5 = confusion_matrix(test_target, preds5)

from sklearn.metrics import f1_score

f1_weighted1 = f1_score(test_target, preds1, average = 'weighted')
f1_weighted2 = f1_score(test_target, preds2, average = 'weighted')
f1_weighted3 = f1_score(test_target, preds3, average = 'weighted')
f1_weighted4 = f1_score(test_target, preds4, average = 'weighted')
f1_weighted5 = f1_score(test_target, preds5, average = 'weighted')

f1_macro1 = f1_score(test_target, preds1, average = 'macro')
f1_macro2 = f1_score(test_target, preds2, average = 'macro')
f1_macro3 = f1_score(test_target, preds3, average = 'macro')
f1_macro4 = f1_score(test_target, preds4, average = 'macro')
f1_macro5 = f1_score(test_target, preds5, average = 'macro')

f1_micro1 = f1_score(test_target, preds1, average = 'micro')
f1_micro2 = f1_score(test_target, preds2, average = 'micro')
f1_micro3 = f1_score(test_target, preds3, average = 'micro')
f1_micro4 = f1_score(test_target, preds4, average = 'micro')
f1_micro5 = f1_score(test_target, preds5, average = 'micro')

f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)
f1_score_weighted5 = "%.2f"%round(f1_weighted5, 2)

f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
f1_score_macro4 = "%.2f"%round(f1_macro4, 2)
f1_score_macro5 = "%.2f"%round(f1_macro5, 2)

f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
f1_score_micro4 = "%.2f"%round(f1_micro4, 2)
f1_score_micro5 = "%.2f"%round(f1_micro5, 2)

head = PrettyTable(['random_state', 'activation', 'solver', 'hidden_layer_sizestuple', 'alpha', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro']) 
head.add_row(['', '', '', '', '', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1]) 
head.add_row(['10', 'tanh', 'lbfgs', '(5, 2)', '0.01', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2]) 
head.add_row(['1', 'logistic', 'sgd', '(150, 100, 50)', 'le-5', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3]) 
head.add_row(['90', 'relu', 'adam', '(100, 50, 30)', '0.05', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4]) 
head.add_row(['40', 'identity', 'lbfgs', '(120, 80, 40)', '0.0001', cr5, f1_score_weighted5, f1_score_macro5, f1_score_micro5]) 
print(head)

Multi layer perceptron neural network
+--------------+------------+--------+-------------------------+--------+------------------+-------------------+----------------+----------------+
| random_state | activation | solver | hidden_layer_sizestuple | alpha  | confusion_matrix | f1_score_weighted | f1_score_macro | f1_score_micro |
+--------------+------------+--------+-------------------------+--------+------------------+-------------------+----------------+----------------+
|              |            |        |                         |        |    [[123   6]    |        0.81       |      0.56      |      0.84      |
|              |            |        |                         |        |    [ 18   3]]    |                   |                |                |
|      10      |    tanh    | lbfgs  |          (5, 2)         |  0.01  |    [[129   0]    |        0.80       |      0.46      |      0.86      |
|              |            |        |                         |        |    [ 2

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Support vector machine

In [ ]:
print('Support vector machine')
from sklearn.svm import SVC

'''Create Artificial Neural Network classifier'''
model1 = SVC()
model2 = SVC(kernel='sigmoid', C=100, gamma=1, random_state=85)
model3 = SVC(kernel='rbf', C=0.1, gamma=0.01, random_state=15)
model4 = SVC(kernel='linear', C=1, gamma=0.001, random_state=74)

'''Train and test'''
model1.fit(train_features, train_target)
model2.fit(train_features, train_target)
model3.fit(train_features, train_target)
model4.fit(train_features, train_target)

preds1 = model1.predict(test_features)
preds2 = model2.predict(test_features)
preds3 = model3.predict(test_features)
preds4 = model4.predict(test_features)

cr1 = confusion_matrix(test_target, preds1)
cr2 = confusion_matrix(test_target, preds2)
cr3 = confusion_matrix(test_target, preds3)
cr4 = confusion_matrix(test_target, preds4)

from sklearn.metrics import f1_score

f1_weighted1 = f1_score(test_target, preds1, average = 'weighted')
f1_weighted2 = f1_score(test_target, preds2, average = 'weighted')
f1_weighted3 = f1_score(test_target, preds3, average = 'weighted')
f1_weighted4 = f1_score(test_target, preds4, average = 'weighted')

f1_macro1 = f1_score(test_target, preds1, average = 'macro')
f1_macro2 = f1_score(test_target, preds2, average = 'macro')
f1_macro3 = f1_score(test_target, preds3, average = 'macro')
f1_macro4 = f1_score(test_target, preds4, average = 'macro')

f1_micro1 = f1_score(test_target, preds1, average = 'micro')
f1_micro2 = f1_score(test_target, preds2, average = 'micro')
f1_micro3 = f1_score(test_target, preds3, average = 'micro')
f1_micro4 = f1_score(test_target, preds4, average = 'micro')

f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)

f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
f1_score_macro4 = "%.2f"%round(f1_macro4, 2)

f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
f1_score_micro4 = "%.2f"%round(f1_micro4, 2)

head = PrettyTable(['random_state', 'c', 'gamma', 'kernel', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro']) 
head.add_row(['', '', '', '', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1]) 
head.add_row(['85', '100', '1', 'sigmoid', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2]) 
head.add_row(['15', '0.01', '0.01', 'rbf', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3]) 
head.add_row(['74', '1', '0.001', 'linear', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4]) 
print(head)

Support vector machine
+--------------+------+-------+---------+------------------+-------------------+----------------+----------------+
| random_state |  c   | gamma |  kernel | confusion_matrix | f1_score_weighted | f1_score_macro | f1_score_micro |
+--------------+------+-------+---------+------------------+-------------------+----------------+----------------+
|              |      |       |         |    [[129   0]    |        0.80       |      0.46      |      0.86      |
|              |      |       |         |    [ 21   0]]    |                   |                |                |
|      85      | 100  |   1   | sigmoid |    [[129   0]    |        0.80       |      0.46      |      0.86      |
|              |      |       |         |    [ 21   0]]    |                   |                |                |
|      15      | 0.01 |  0.01 |   rbf   |    [[129   0]    |        0.80       |      0.46      |      0.86      |
|              |      |       |         |    [ 21   0]]  

decision tree

In [ ]:
print('Decision tree')
from sklearn import tree

'''Split into train and test with numpy (0.75 train, 0.25 test)'''
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.25)

head = PrettyTable(['random_state', 'max_depth', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro']) 
  
max_depth_range = list(range(1, 15))
for depth in max_depth_range:

  '''Create decision tree classifier'''    
  model = tree.DecisionTreeClassifier(max_depth = depth,  random_state = 90)
  
  '''Train and test'''
  model.fit(train_features, train_target)
  preds = model.predict(test_features)

  cr = confusion_matrix(test_target, preds)

  from sklearn.metrics import f1_score

  f1_weighted = f1_score(test_target, preds, average = 'weighted')
  f1_macro = f1_score(test_target, preds, average = 'macro')
  f1_micro = f1_score(test_target, preds, average = 'micro')

  f1_score_weighted = "%.2f"%round(f1_weighted, 2)
  f1_score_macro = "%.2f"%round(f1_macro, 2)
  f1_score_micro = "%.2f"%round(f1_micro, 2)

  head.add_row(['90', depth, cr, f1_score_weighted, f1_score_macro, f1_score_micro])
print(head)

Decision tree
+--------------+-----------+------------------+-------------------+----------------+----------------+
| random_state | max_depth | confusion_matrix | f1_score_weighted | f1_score_macro | f1_score_micro |
+--------------+-----------+------------------+-------------------+----------------+----------------+
|      90      |     1     |    [[129   0]    |        0.80       |      0.46      |      0.86      |
|              |           |    [ 21   0]]    |                   |                |                |
|      90      |     2     |    [[129   0]    |        0.80       |      0.46      |      0.86      |
|              |           |    [ 21   0]]    |                   |                |                |
|      90      |     3     |    [[128   1]    |        0.81       |      0.51      |      0.86      |
|              |           |    [ 20   1]]    |                   |                |                |
|      90      |     4     |    [[128   1]    |        0.81       | 